In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Duchenne_Muscular_Dystrophy/GSE48828'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression and Splicing Analysis of Myotonic Dystrophy and Other Dystrophic Muscle"
!Series_summary	"The prevailing patho-mechanistic paradigm for myotonic dystrophy (DM) is that the aberrant presence of embryonic isoforms is responsible for many, if not most, aspects of the pleiotropic disease phenotype.  In order to identify such aberrantly expressed isoforms in skeletal muscle of DM type 1 (DM1) and type 2 (DM2) patients, we utilized the Affymetrix exon array to characterize the largest collection of DM samples analyzed to date, and included non-DM dystrophic muscle samples (NMD) as disease controls."
!Series_overall_design	"For the exon array profiling on the Human Exon 1.0 ST array (Affymetrix Santa Clara, CA) we used a panel of 28 skeletal muscle biopsies from DM1 (n=8), DM2 (n=10), Becker muscular dystrophy, BMD, (n=3), Duchenne muscular dystrophy, DMD (n=1), Tibial muscular dystrophy, TMD, (n=2) and normal skeletal muscle (n=4).  Normal co

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Define sample_characteristics
sample_characteristics = {
    0: ['diagnosis: Myotonic Dystrophy Type 1', 'diagnosis: Myotonic Dystrophy Type 2', 'diagnosis: Becker Muscular Dystrophy', 'diagnosis: Duchenne Muscular Dystrophy', 'diagnosis: Tibial muscular Dystophy', 'diagnosis: Normal'],
    1: ['gender: F', 'gender: M', 'gender: Not available'],
    2: ['age (yrs): Not available', 'age (yrs): 54', 'age (yrs): 29', 'age (yrs): 25', 'age (yrs): 21', 'age (yrs): 55', 'age (yrs): na', 'age (yrs): 39', 'age (yrs): 58', 'age (yrs): 50', 'age (yrs): 51', 'age (yrs): 43', 'age (yrs): 37', 'age (yrs): 65', 'age (yrs): 45', 'age (yrs): 26', 'age (yrs): 20', 'age (yrs): 88', 'age (yrs): 61', 'age (yrs): 85']
}

# Check if gene expression data is available
is_gene_available = True  # Assume gene expression data is available from the Affymetrix exon array

# Data availability and type conversion
trait_row = 0 if 'Duchenne Muscular Dystrophy' in {info.split(': ')[1] for info in sample_characteristics[0]} else None
age_row = 2 if len({info.split(': ')[1] for info in sample_characteristics[2] if not info.endswith('Not available') and not info.endswith('na')}) > 1 else None
gender_row = 1 if len({info.split(': ')[1] for info in sample_characteristics[1] if info.split(': ')[1] != 'Not available'}) > 1 else None

# Define conversion functions
def convert_trait(value):
    val = value.split(": ")[1]
    return 1 if val == 'Duchenne Muscular Dystrophy' else 0 if val == 'Normal' else None

def convert_age(value):
    val = value.split(": ")[1]
    try:
        return float(val) if val != 'Not available' and val != 'na' else None
    except ValueError:
        return None

def convert_gender(value):
    val = value.split(": ")[1]
    return 0 if val == 'F' else 1 if val == 'M' else None

# Save cohort information
save_cohort_info('GSE48828', './preprocessed/Duchenne_Muscular_Dystrophy/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Duchenne_Muscular_Dystrophy', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Duchenne_Muscular_Dystrophy/trait_data/GSE48828.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1185313': [None, None, 0], 'GSM1185314': [None, None, 0], 'GSM1185315': [nan, 54.0, 0.0], 'GSM1185316': [nan, 29.0, 0.0], 'GSM1185317': [nan, 25.0, 1.0], 'GSM1185318': [nan, 21.0, 1.0], 'GSM1185319': [nan, 55.0, 0.0], 'GSM1185320': [None, None, None], 'GSM1185321': [nan, 39.0, 1.0], 'GSM1185322': [nan, 58.0, 0.0], 'GSM1185323': [nan, 50.0, 0.0], 'GSM1185324': [nan, 51.0, 0.0], 'GSM1185325': [nan, 43.0, 0.0], 'GSM1185326': [nan, 51.0, 1.0], 'GSM1185327': [nan, 37.0, 0.0], 'GSM1185328': [nan, 43.0, 0.0], 'GSM1185329': [nan, 65.0, 0.0], 'GSM1185330': [nan, 55.0, 0.0], 'GSM1185331': [nan, 50.0, 1.0], 'GSM1185332': [nan, 45.0, 1.0], 'GSM1185333': [nan, 26.0, 1.0], 'GSM1185334': [1.0, 20.0, 1.0], 'GSM1185335': [nan, 58.0, 1.0], 'GSM1185336': [nan, 88.0, 0.0], 'GSM1185337': [0.0, 61.0, 0.0], 'GSM1185338': [0.0, 43.0, 1.0], 'GSM1185339': [0.0, 85.0, 1.0], 'GSM1185340': [0.0, 43.0, 1.0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for identifiers and gene symbols in the dictionary
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Duchenne_Muscular_Dystrophy/gene_data/GSE48828.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Duchenne_Muscular_Dystrophy')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE48828', './preprocessed/Duchenne_Muscular_Dystrophy/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Duchenne_Muscular_Dystrophy/GSE48828.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Duchenne_Muscular_Dystrophy', the least common label is '1.0' with 1 occurrences. This represents 20.00% of the dataset.
The distribution of the feature 'Duchenne_Muscular_Dystrophy' in this dataset is fine.

Quartiles for 'Age':
  25%: 43.0
  50% (Median): 43.0
  75%: 61.0
Min: 20.0
Max: 85.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 1 occurrences. This represents 20.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

